TRY
top_k_categorical_accuracy
https://stackoverflow.com/questions/47887533/keras-convolution-along-samples
https://keras.io/layers/wrappers/#timedistributed

## This file differs from 'training_simple_single' by new data generator

In [1]:
input_width = 160
input_height = 100
channels = 3
class_number = 12
data_path = "D:\\Python\\Keras\\Wormax\\data_prepared\\"
model_name = 'models/worm_single_xceptiondelme.h5'
look_forward = 1
use_rotation = False
import keras

Q:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers, models, optimizers
import keras.backend as K

def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))

def define_model():
    '''model = models.Sequential()
        
    model.add(layers.Convolution2D(64, (8, 8), strides=(4, 4), activation='relu',
                            input_shape=(input_height, input_width, channels)))
    model.add(layers.Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(layers.Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(class_number, activation='softmax'))    '''
    model = keras.applications.mobilenetv2.MobileNetV2(include_top=True, 
                                                       weights=None, 
                                                       input_tensor=None, 
                                                       input_shape=(input_height, input_width, channels), 
                                                       pooling=None, 
                                                       classes=12)

    model.compile(optimizer=optimizers.Adam(lr=1e-3),
                  loss='categorical_crossentropy',
                  metrics=[actual_acc])
    
    model.summary()
    return model

In [3]:
model = define_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 50, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 50, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 50, 80, 32)   0           bn_Conv1[0][0]                   
__________________________________________________________________________________________________
expanded_c

In [4]:
# little prepocessing
from math import atan2, pi
import cv2

def get_angle(x, y):
    return atan2(y, x)

def get_direction(x, y, n_classes = 12):
    return round(get_angle(x, y)/2/pi*n_classes)%n_classes

# Rotating frame and direction counter-clockwise
def get_rotated_frame(img, rotation, target_direction, n_classes):
    if rotation != 0:
        rows, cols = len(img),len(img[0])
        rot_M = cv2.getRotationMatrix2D((cols/2, rows/2), rotation*360/n_classes, 1)
        img = cv2.warpAffine(img, rot_M, (cols, rows))
        target = (target_direction - rotation)%n_classes
    return img, target_direction

In [5]:
import numpy as np
import os
import random
from functools import reduce
from keras.utils import to_categorical

# Training and validation
data_ratio = 0.7

def generator(data_dir, n_classes, role, shuffle=True, batch_size=128):
    
    listdir = []
    listdir = filter(lambda x: os.path.isfile, os.listdir(data_dir))
    listdir = np.array(list(listdir))
    random.shuffle(listdir)
    
    #print('Found {} files for {}'.format(len(listdir), role))
    
    file_i = 0
    while 1:
        arr = np.load(data_dir + listdir[file_i])        
        file_i = (file_i+1) if file_i+1<len(listdir) else 0
        
        # Expanding blocks
        data = []        
        for i in arr:
            for j in i:
                data.append(j)
        data = np.array(data)

        
        if role == 'train':
            data = data[:int(round(len(data)*data_ratio))]
        elif role == 'validation':
            data = data[int(round(len(data)*data_ratio)):]
        else:
            raise 'bad role parameter'
        
        # Generating y
        data_targets = np.zeros((len(data)-look_forward))
        for i in range(len(data_targets)):
            data_targets[i] = get_direction(*data[i+look_forward][1][:2])
            #data_targets[i] = np.array(to_categorical(get_direction(*data[i+look_forward][1][:2]), num_classes=num_classes))
        data = data[:len(data)-look_forward]
        
        # Only X
        data_features = np.zeros((len(data), input_height, input_width, channels))
        for i in range(len(data)):
            data_features[i] = data[i][0][0]
        
        indexes = np.arange(len(data_features)-look_forward)        
        if shuffle:
            np.random.shuffle(indexes)
        
        # Rotating cycle(no rotation for validation)
        rotate_times = n_classes if role == 'train' and use_rotation else 1
        for rot_i in range(rotate_times):
            # Batches forming cycle
            for i in range(0, len(data)-look_forward-batch_size, batch_size):
                samples = data_features.take(indexes[i:i+batch_size], axis=0)
                targets = data_targets.take(indexes[i:i+batch_size], axis=0)
                samples_rot = np.zeros_like(samples)
                targets_rot = np.zeros((batch_size, n_classes))
                
                # Rotating
                for j in range(batch_size):
                    if use_rotation:
                        rotation = (rot_i+j+i)%n_classes
                        img, target = get_rotated_frame(samples[j], rotation, targets[j], n_classes)
                    else:
                        img, target = samples[j], targets[j]
                    samples_rot[j] = img
                    targets_rot[j] = to_categorical(target, num_classes=n_classes)
                    
                # Normilize
                samples_rot = samples_rot / 255
                yield samples_rot, targets_rot

In [6]:
import numpy as np

train_generator = generator(data_path, class_number, 'train', batch_size=16)
validation_generator = generator(data_path, class_number, 'validation', batch_size=32)

print(next(train_generator)[0].shape)
print(next(train_generator)[1].shape)

(16, 100, 160, 3)
(16, 12)


In [7]:
# count class instances count for balancing
if False:
    i = 0
    classes = np.zeros((class_number))
    for samples, targets in generator(data_path, class_number, 'train', batch_size=1024):
        for j in targets:
            classes += j
        i += 1
        if i == 100:
            break
    print(classes)

### tensorboard --logdir=D:\Python\Keras\Wormax\log_dir

# TODO
    2) tune look_forward
    3) change target to angle
    4) fight overfitting on adam
    5) Take away poolingb

I am trying to solve classification task. Basically, teaching game bot on human actions.  
So there are more than 1 right answers for X. But my dataset contains specific actions.

Assume the desirable answer is:`[0, 0.7, 0, 0.7, 0.1]`  
But dataset contains one-hots like: `[0, 0, 0, 1, 0]`

So my idea is to implement Q-state neural network with 2 inputs.  
Q(S,a) - where, S - current state, a - action, result is one number(desire to choose this action).  
And then I can feed to it my X->y dataset.

I am struggling with initializing model. Default


In [8]:
initial_epoch = 0
if not True:
    model.load_weights(model_name)
    initial_epoch = 176

In [ ]:
if not use_rotation:
    class_weight = {0: 1.15,
                     1: 1.13,
                     2: 1.27,
                     3: 1.51,
                     4: 1.46,
                     5: 1.19,
                     6: 1.0,
                     7: 1.04,
                     8: 1.11,
                     9: 1.2,
                     10: 1.27,
                     11: 1.34}

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log_dir\\' + model_name
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name,
        monitor='val_loss',
        save_best_only=True,
    )
    
]
'''keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        factor=0.5,                              
        patience=70, 
        min_lr=0.00001
    )'''

steps_per_epoch = 100
history = model.fit_generator(train_generator,
                            steps_per_epoch=steps_per_epoch,
                            epochs=500,
                            validation_data=validation_generator,
                            validation_steps=int(round(steps_per_epoch/data_ratio*(1-data_ratio))),
                            shuffle=True,
                            class_weight=class_weight,
                            callbacks=callbacks,
                            initial_epoch=initial_epoch
                        )

Epoch 1/500
100/100 [==============================] - 24s 244ms/step - loss: 3.2082 - actual_acc: 0.0981 - val_loss: 7.3228 - val_actual_acc: 0.0661
Epoch 2/500
100/100 [==============================] - 17s 175ms/step - loss: 3.0732 - actual_acc: 0.1175 - val_loss: 6.6997 - val_actual_acc: 0.0683
Epoch 3/500
100/100 [==============================] - 20s 202ms/step - loss: 2.9578 - actual_acc: 0.1556 - val_loss: 7.8099 - val_actual_acc: 0.0894
Epoch 4/500
100/100 [==============================] - 18s 184ms/step - loss: 2.8573 - actual_acc: 0.1781 - val_loss: 7.1448 - val_actual_acc: 0.0850
Epoch 5/500
100/100 [==============================] - 31s 311ms/step - loss: 2.6135 - actual_acc: 0.2575 - val_loss: 9.3119 - val_actual_acc: 0.0734
Epoch 6/500
100/100 [==============================] - 18s 179ms/step - loss: 2.3327 - actual_acc: 0.3450 - val_loss: 9.4745 - val_actual_acc: 0.0908
Epoch 7/500
100/100 [==============================] - 21s 205ms/step - loss: 2.1687 - actual_acc: 0

100/100 [==============================] - 18s 179ms/step - loss: 1.9482 - actual_acc: 0.4325 - val_loss: 6.6079 - val_actual_acc: 0.1272
Epoch 56/500
100/100 [==============================] - 27s 265ms/step - loss: 1.6844 - actual_acc: 0.4931 - val_loss: 5.5222 - val_actual_acc: 0.1410
Epoch 57/500
100/100 [==============================] - 18s 179ms/step - loss: 1.5324 - actual_acc: 0.5337 - val_loss: 4.7149 - val_actual_acc: 0.1446
Epoch 58/500
100/100 [==============================] - 28s 275ms/step - loss: 1.6627 - actual_acc: 0.5044 - val_loss: 8.2303 - val_actual_acc: 0.1119
Epoch 59/500
100/100 [==============================] - 19s 189ms/step - loss: 1.9918 - actual_acc: 0.3981 - val_loss: 4.9929 - val_actual_acc: 0.1541
Epoch 60/500
100/100 [==============================] - 18s 179ms/step - loss: 1.6403 - actual_acc: 0.5031 - val_loss: 4.6097 - val_actual_acc: 0.1315
Epoch 61/500
100/100 [==============================] - 23s 226ms/step - loss: 1.5746 - actual_acc: 0.5369 

In [ ]:
from keras import backend as K
K.clear_session()

# Урок
    1) автоматизируйте подбор гиперпараметров
    2) baseline